In [2]:
import optuna
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, recall_score

In [3]:
def dataset(partition, feature_set):
    train = pd.read_csv(os.path.join('./data_csv/',partition,feature_set+'_train.csv'))
    valid = pd.read_csv(os.path.join('./data_csv/',partition,feature_set+'_devel.csv'))
    return train, valid

In [4]:
def f1(y_true, y_pred):
    return round(f1_score(y_true, y_pred, average='micro') * 100, 3)

def uar(y_true, y_pred):
    return round(recall_score(y_true, y_pred, average='macro') * 100, 3)

def combine(y_true, y_pred):
    return round((0.66 * f1(y_true, y_pred) + 0.34 * uar(y_true, y_pred)), 3)

partition = 'arousal'
feature_set = 'vggface'
train, valid = dataset(partition, feature_set)
last_columns = '511'

In [ ]:
def ensemble(partition, feature_set, last_columns):
    
    X_train, y_train = train.loc[:,'segment_id':'{}'.format(last_columns)], train['class_id']
    X_valid, y_valid = valid.loc[:,'segment_id':'{}'.format(last_columns)], valid['class_id']
    
    

    
    model1 = SVC(C = 1.0235576771757582, gamma = 'auto', probability=True)
    model1.fit(X_train,y_train)
    pred1 = model1.predict_proba(X_valid)
#     pred1 = model1.predict(X_valid)

    model2 = KNeighborsClassifier(n_neighbors = 683)
    model2.fit(X_train,y_train)
    pred2 = model2.predict_proba(X_valid)
    
    pred = (0.5*pred1+0.5*pred2)
    result = np.argmax(pred, axis=1)

    acc = combine(y_valid, result)
    print(acc)